# Astropy tables: columns and units

In [ ]:
import numpy as np
import astropy.units as u
from astropy.table import Table, QTable
from astropy.coordinates import SkyCoord, Angle
import matplotlib.pyplot as plt

⚠️ `Tables` with units have limited functionality. For full unit management, it is recommended to use `QTable`.

In [ ]:
a = np.array([1, 4, 5], dtype=np.int32)
b = [2.0, 5.0, 8.5] * u.cm
c = ['x', 'y', 'z']
d = [10, 20, 30] * u.m / u.s

t = QTable([a, b, c, d],
           names=('id', 'length', 'label', 'velocity'),
           meta={'name': 'first table'})

In [ ]:
t

In [ ]:
t['velocity']

In [ ]:
t['velocity'][1]

In [ ]:
t['velocity'][0:2]

In [ ]:
t[2]

In [ ]:
t['velocity'].unit

## Obtaining information and statistics

In [ ]:
t.info

In [ ]:
t.info('stats')

In [ ]:
stats = t.info('stats', out=None)
stats

In [ ]:
stats[3]['mean']

## Time and coordinates

In [ ]:
from astropy.time import Time
from astropy.coordinates import SkyCoord

Time.FORMATS

In [ ]:
mjd = Time([56200.25, 56400.33, 57500.66], format='mjd', scale='utc')
date = mjd.to_datetime()
sc = SkyCoord([10, 20, 30], [-45, +40, +55], unit='deg')
tab = QTable([mjd, date,  sc, sc.to_string('hmsdms')], names=['MJD', 'Date', 'skycoord', 'coord'])
tab

In [ ]:
tab['coord']

## Managing table columns

In [ ]:
tab.columns

In [ ]:
tab.colnames

Substitute column

In [ ]:
tab['MJD'] = [50000, 60000, 70000]
tab

Create new column

In [ ]:
tab['x'] = [1,2,3]
tab['y'] = [10,20,30]
tab

In [ ]:
tab['z'] = tab['x']*2 + tab['y']
tab

Selection of relevant columns

In [ ]:
tab['x']

In [ ]:
type(tab['x'])

In [ ]:
tab['x'].data

In [ ]:
tab['x'].mean()


In [ ]:
tab[['MJD', 'x', 'z']]


In [ ]:
my_columns = ['x', 'y', 'z']
tab[my_columns]

## Load our problem data from last session

In [ ]:
data0 = Table.read('../data/data0.ecsv')

### ⛏ Exercise
- Print the first 5 rows of the table with `data0[0:5]`.
- Obtain a list of columns of the table `data0`.
- Create a new table `my_table` containing only the columns Right Ascension, Declination, and the two proper motions, and the associated errors for all of them. You should obtain a table with 8 columns.
- Obtain the description (column name, format, units and description) using .info() method.
- Print the mean R.A. in degress and the median Declination in degrees.
- Print the standard deviation (`np.std`) of the R.A. in arcseconds and of the Declination in arcmin.
- Compute the minimum and maximum proper motion in right ascension in units of mas/yr
- Compute the minimum and maximum proper motion in declination in units of arcmin/day
- Create a new column `pos_error` in `my_table` with the total uncertainty in the position that accounts for the quadratic sum (`np.sqrt(()**2 + ()**2)`) of the uncertainty of the R.A. and Declination components. Express in units of deg.
- Create a new column `pm_error` in `my_table` with the total uncertainty in the position that accounts for the quadratic sum (`np.sqrt(()**2 + ()**2)`) of the uncertainty of the proper motion components in units of arcmin per minute.


In [ ]:
data0[0:5]

In [ ]:
data0.columns

In [ ]:
my_columns = ['ra', 'ra_error', 'dec', 'dec_error', 'pmra', 'pmdec', 'pmra_error', 'pmdec_error']
my_table = data0[my_columns]
my_table

In [ ]:
#type(QTable(my_table))
#my_table = QTable(my_table)

In [ ]:
my_table.info()

In [ ]:
my_table['ra'].mean(), my_table['dec'].mean()

In [ ]:
np.std(my_table['ra'].to(u.arcsec)), np.std(my_table['dec'].to(u.arcmin))

In [ ]:
np.min(my_table['pmra']), np.max(my_table['pmra'])

In [ ]:
np.min(my_table['pmdec'].to(u.arcmin/u.day)), np.max(my_table['pmdec'].to(u.arcmin/u.day))

In [ ]:
my_table['pos_error'] = np.sqrt(my_table['ra_error']**2 + my_table['dec_error']**2).to('deg')
my_table[0:3]

In [ ]:
pos_error = np.sqrt(my_table['pmra_error']**2 + my_table['pmdec_error']**2)
my_table['pos_error'] = pos_error.to(u.arcmin/u.min)
my_table[0:3]

### ⛏ Exercise
Compute absolute magnitude

In [ ]:
data0['Mg'] = data0['phot_g_mean_mag']+5*np.log10(data0['parallax'])-10
data0[0:3]

# Exploratory analysis

In [ ]:
import matplotlib.pyplot as plt
from astropy.visualization import quantity_support

In [ ]:
plt.plot(data0['ra'], data0['dec'], '.', color='k', alpha = 0.1, ms=1)

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(15,10))
ax.scatter(data0['ra'], data0['dec'], s=data0['phot_g_mean_flux']/1e5);
ax.set_aspect('equal')

# Here we invert the direction of the right ascension axis
ax.invert_xaxis()

ax.set_xlabel('Right Ascension [deg]')
ax.set_ylabel('Declination [deg]');

In [ ]:
plt.plot(data0['pmra'], data0['pmdec'], marker='.', ls='', color='k');

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(12,8))
ax.plot(data0['phot_g_mean_flux'], data0['parallax_error'], '.k', ms=1, alpha=0.5)
ax.loglog();

Never forget to add labels to identify what you are plotting!

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(12,8))
ax.plot(data0['phot_g_mean_flux'], data0['parallax_error'], '.k', ms=1, alpha=0.5)
ax.loglog()

ax.set_xlabel('G-band mean flux [e/s]')
ax.set_ylabel('Parallax error [mas]');

In [ ]:
def str_label(tab, col):
    return f"{tab[col].description} [{tab[col].quantity.unit}]"

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(12,8))
col1 = 'phot_g_mean_flux'
col2 = 'parallax_error'

ax.plot(data0[col1], data0[col2], '.k', ms=1, alpha=0.5)

ax.loglog()

ax.set_xlabel(str_label(data0, col1))
ax.set_ylabel(str_label(data0, col2));

In [ ]:
def plot_variables(data, col1, col2, loglog=True, **kwargs):
    fig, ax = plt.subplots(ncols=1, figsize=(12,8))

    ax.plot(data[col1], data[col2], **kwargs)
    if loglog:
        ax.loglog()
    ax.set_xlabel(str_label(data0, col1))
    ax.set_ylabel(str_label(data0, col2));

    return fig, ax

In [ ]:
fig, ax = plot_variables(data0, col1='pmra', col2='pmra_error', marker='.', color='k', ms=2, alpha=0.5, ls='')

In [ ]:
fig, ax = plot_variables(data0, col1='phot_g_mean_mag', col2='parallax', marker='.', color='k', ms=2, alpha=0.5, ls='')

### 🌪 Exercise
Define a fucntion similar to `plot_variables` that can be used to plot 3 columns against each other in three different subplots.

## Coordinate plots in sky projections

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(data0['ra'].to(u.radian), data0['dec'].to(u.radian), marker='.', color='r')
ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
ax.grid(True)

Let's overplot the galactic plane

In [ ]:
galactic_plane = SkyCoord(l=np.arange(-180, 180), b=np.zeros(360), frame='galactic', unit=u.deg)
galactic_plane_eq = galactic_plane.transform_to('icrs')
gal_ra  = galactic_plane_eq.ra.wrap_at('180d').radian
gal_dec = galactic_plane_eq.dec.wrap_at('180d').radian

fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection="mollweide")
ax.scatter(data0['ra'].to(u.radian), data0['dec'].to(u.radian), marker='.', color='r')
plt.plot(gal_ra, gal_dec, 'k.')
ax.set_xticklabels(['14h','16h','18h','20h','22h','0h','2h','4h','6h','8h','10h'])
ax.grid(True)

### 🌪 Exercise
Make the previous plot but in Galactic coordinates

### 🌪 Exercise
Prepare a plot `pmra` vs `pmdec` including errorbars using the matplotlib function `plt.errorbars`. The key point is to remove NaNs from the table. Spoiler alert: there is no easy way to do this in astropy `Table`, apparently, since [this PR](https://github.com/astropy/astropy/issues/7446) was never finished.

In [ ]:
# Astropy doesn't seem to have an easy way to remove rows with NaNs https://github.com/astropy/astropy/issues/7446
# I will use a bad trick of converting to pandas. Note we lose units information!
pm = data0[['pmra','pmdec','pmra_error', 'pmdec_error']]
pm = Table.from_pandas(pm.to_pandas().dropna())

plt.errorbar(pm['pmra'], pm['pmdec'],
             yerr=pm['pmra_error'],
             xerr=pm['pmdec_error'],
             marker='.', ls='', color='k', alpha=0.4)
plt.xlim(-50, 50)
plt.ylim(-50, 50)

## Histogram distributions

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(15,10))
ax.hist(data0['parallax'], bins=np.arange(-5, 15, 0.5))

ax.set_xlabel('Parallax [mas]')
ax.set_ylabel('Number of stars');

### ⛏ Exercise
Modify the line `ax.set_xlabel('Parallax [mas]')` to automatically find the units of the column being plotted.

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(15,10))

ax.hist(data0['parallax'], bins=np.arange(-5, 15, 0.5))

description = data0['parallax'].info(out=None)['description']
unit = data0['parallax'].info(out=None)['unit']
ax.set_xlabel(f"{description} [{unit}]")
ax.set_ylabel('Number of stars');

First of all we see that there are negative parallaxes, which don't have physical meaning, but are a consequence of errors in the parallax determination. We can create a second `QTable` that ignores any negative parallax


In [ ]:
positive_parallaxes = data0['parallax'] > 0
data1 = data0[positive_parallaxes]
print(len(data0))
print(len(data1))

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(15,10))
ax.hist(data0['parallax'], bins=np.arange(-5, 15, 0.5), color='grey')
ax.hist(data1['parallax'], bins=np.arange(-5, 15, 0.5))
    
description = data0['parallax'].info(out=None)['description']
unit = data0['parallax'].info(out=None)['unit']
ax.set_xlabel(f"{description} [{unit}]")
ax.set_ylabel('Number of stars');

Let's work with distances in kpc, that are more familiar to us. We will use an astropy unit transformation as before. However, an angle (mas) cannot be converted to distance (kpc) without knowing how the transformation should occur. We need to parse which equivalency to use to make the transormation.

There is a lot of information in [Equivalencies](https://docs.astropy.org/en/stable/units/equivalencies.html), for example to convert spectral units (nm to Hz) or conversions from wavelength/frequency/energy including doppler effect.

In [ ]:
data1['distance'] = data1['parallax'].to(u.kpc, equivalencies=u.parallax())
data1['distance'].description = 'Distance from Earth'
data1[0:3]

### ⛏ Exercise
Compute yourself the distance in kpc and check that the transformation has worked. Start by using the numpy array without units `np.array(data1['parallax'])` and transform it with the parallax formula $d [{\rm kpc}]= \frac{1}{p[{\rm mas}]}$, where $p$ is the parallax.

In [ ]:
# We compute the ditance as 1/parallax. When the parallax is in mas, the distance will be in kpc.
p = np.array(data1['parallax']) # Let's get rid of the units because we want to do the conversion manually
d = 1./p
d_kpc = u.Quantity(d, unit='kpc') # Now we set the correct units back
print(f"Astropy distances: {data1['distance'].data}")
print(f"My distances: {d_kpc}")

# We compute how different are the astropy parallax from equivalence and our manually computed distances:
residual = data1['distance'] - d_kpc
print(f"Mean residual {np.nanmean(residual)}")

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(15,10))
with quantity_support():
    ax.hist(data1['parallax'], bins=np.arange(-0.5, 15, 0.5))

ax.set_xlabel(str_label(data0, 'parallax'))
ax.set_ylabel('Number of stars');

Now we are going to use a finer bin resolution and plot two panels to visualize the parallax and the distance at the same time.

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(18,6))
with quantity_support():
    ax[0].hist(data1['parallax'], bins=np.arange(0, 8, 0.05), label='Parallax distribution')
    ax[1].hist(data1['distance'], bins=np.arange(0, 8, 0.05), label='Distance distribution') # Try 0.01 kpc steps

ax[0].legend()
ax[1].legend()
ax[0].set_xlabel('Parallax [mas]')
ax[0].set_ylabel('Number of stars');
ax[1].set_xlabel('Distance [kpc]')
ax[1].set_ylabel('Number of stars');

We see a very interesting accummulation of stars at a parallax of approximately 5.2 mas. We can create a filter to select the start in that particular range. We will overplot the distribution of the whole sample and the one of the selected group.

In [ ]:
manual_filter1 = (data1['parallax'] > 5.0*u.mas) & (data1['parallax'] < 5.7*u.mas)
cluster1 = data1[manual_filter1]
cluster1

Theres is an alternative way to deal with this problem, and it is to create a new column that can be True if the star is part of the cluster of False if not

In [ ]:
data1['cluster'] = manual_filter1
data1[0:3]

The cluster is simply selecting the rows matching the filter = selecting the true values

In [ ]:
data1[manual_filter1]

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(18,6))
with quantity_support():
    ax[0].hist(data1['parallax'],    bins=np.arange(0, 8, 0.05), label='Full sample')
    ax[0].hist(cluster1['parallax'], bins=np.arange(0, 8, 0.05), label='Cluster')

    ax[1].hist(data1['distance'],    bins=np.arange(0, 2, 0.01), label='Full sample')
    ax[1].hist(cluster1['distance'], bins=np.arange(0, 2, 0.01), label='Cluster')

ax[0].legend()
ax[1].legend()
ax[0].set_xlabel('Parallax [mas]')
ax[0].set_ylabel('Number of stars');
ax[1].set_xlabel('Distance [kpc]')
ax[1].set_ylabel('Number of stars');

ax[0].legend()
ax[1].legend();

In [ ]:
fig, ax = plot_variables(data1, col1='phot_g_mean_mag', col2='parallax', marker='.', color='k', ms=2, alpha=0.5, ls='')

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(12,8))
col1 = 'phot_g_mean_flux'
col2 = 'parallax_error'

ax.plot(data1['phot_g_mean_mag'], data1['parallax'],'.k', ms=1, alpha=0.5)
ax.plot(cluster1['phot_g_mean_mag'], cluster1['parallax'], marker='o', color='r', ms=2, ls='')

ax.loglog()

ax.set_xlabel(str_label(data0, col1))
ax.set_ylabel(str_label(data0, col2));

We see that the cluster is dominated by bright stars (lower magnitude), and form a sharp cluster in parallax/distance, as we already know.

## Spatial distribution of the cluster
We plot the distribution of start in the sky. First, all the stars in the sample are plotted in grey. The stars of the cluster and plotted in color, with the colorscale representing the distance from the Earth in pc.

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(15,10))
ax.set_aspect('equal')

ax.scatter(data1['ra'], data1['dec'], c='gray', s=1, alpha=0.5);
l = ax.scatter(cluster1['ra'], cluster1['dec'], c=cluster1['distance']*1000., s=20);

ax.set_xlabel('Right Ascension [deg]')
ax.set_ylabel('Declination [deg]');

# Here we invert the direction of the right ascension axis
ax.invert_xaxis()

# Show the color bar
cb = fig.colorbar(l);
cb.set_label('Distance [kpc]')

In [ ]:
fig, ax = plt.subplots(ncols=1, figsize=(15,10))
ax.scatter(data0['ra'], data0['dec'], s=data0['phot_g_mean_flux']/1e5);
ax.scatter(cluster1['ra'], cluster1['dec'], s=cluster1['phot_g_mean_flux']/1e5);

ax.set_aspect('equal')

# Here we invert the direction of the right ascension axis
ax.invert_xaxis()

ax.set_xlabel('Right Ascension [deg]')
ax.set_ylabel('Declination [deg]');

There is no apparent pattern of the selected stars, although there seems to be an overdensity at the center, specially in Right Ascension.

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(18,6))

ra_range = [np.min(data1['ra']), np.max(data1['ra'])]
de_range = [np.min(data1['dec']), np.max(data1['dec'])]
                   
                                       
ax[0].hist(data1['ra'],    bins=np.arange(ra_range[0], ra_range[1], 0.04), label='Full sample')
ax[0].hist(cluster1['ra'], bins=np.arange(ra_range[0], ra_range[1], 0.12), label='Cluster')

ax[1].hist(data1['dec'],    bins=np.arange(de_range[0], de_range[1], 0.02), label='Full sample')
ax[1].hist(cluster1['dec'], bins=np.arange(de_range[0], de_range[1], 0.08), label='Cluster')

ax[0].set_xlabel('Right Ascension [deg]')
ax[0].set_ylabel('Number of stars');
ax[1].set_xlabel('Declination [deg]')
ax[1].set_ylabel('Number of stars');

ax[0].legend()
ax[1].legend();

Nothing obvious or much interesting. The overdensity in the centre for R.A. is clear, but in Dec it is not clear, there may be a slope.

In [ ]:
data1.write('../data/data1.ecsv', format='ascii.ecsv', overwrite=True)
cluster1.write('../data/cluster1.ecsv', format='ascii.ecsv', overwrite=True)

# Pandas tables

In [ ]:
import pandas as pd

In [ ]:
df1 = data1.to_pandas()
df1

In [ ]:
df1.head()

The first thing we see is that we have lost the unit information. That is an important problem if we don't track the column operations properly.

In [ ]:
df1.info()

In [ ]:
df1.columns

In [ ]:
df1['ra']
df1[['ra','dec']]

In [ ]:
df1['pmra'].values

Slicing

In [ ]:
df1.loc[6]

In [ ]:
df1['pmra'].iloc[6]

In [ ]:
df1.iloc[5:8]

In [ ]:
df1[['pmra','pmdec']].iloc[6]

In [ ]:
df1[['pmra','pmdec']].iloc[6:10]

We can easily do operations to columns. First by aggregating values according to some functions

In [ ]:
df1.aggregate(['sum', 'min'])

In [ ]:
df1.aggregate({'ra' : ['mean', 'min', 'max', 'std'],
               'dec' : ['mean', 'min', 'max', 'std'],
               'parallax': 'std'})

In [ ]:
df1.describe()

In [ ]:
def my_func(row):
    return np.sqrt(row['pmra']**2 + row['pmdec']**2)

df1.apply(lambda row: my_func(row), axis=1)

Other cases
- pivot tables
- very powerful groupby
- Complex conditions involving multiple columns

NaN values

In [ ]:
df1['radial_velocity'].isna().sum()

In [ ]:
df_rv = df1['radial_velocity'].dropna()
df_rv

Groupby

In [ ]:
df1['matched_observations'].describe()

In [ ]:
df1.groupby('matched_observations').aggregate(['mean','min'])[['parallax_error','ra_error', 'dec_error']].plot()

Working with datetime series

In [ ]:
N = 100
times = pd.date_range("2021-06-9", periods=N, freq='D')

ts = pd.DataFrame({'v1': np.random.normal(0.5,1,N),
                   'v2': np.random.normal(0.5,1,N)},
                   index=times)
ts

In [ ]:
ts.plot()

Other plots

In [ ]:
df1.plot.scatter(x='phot_g_mean_flux', y='parallax_error', marker='.', alpha=0.1)
plt.loglog()

In [ ]:
df1.plot.scatter(x='phot_g_mean_flux', y='parallax_error',  marker='.', s=0.5, alpha=0.1)
plt.loglog()

In [ ]:
ax.plot(data0['phot_g_mean_flux'], data0['parallax_error'], '.k', ms=1, alpha=0.5)


In [ ]:
df1[['pmra','pmdec']].plot.hist(bins=np.arange(-80, 60, 3), alpha=0.5)


In [ ]:
df1[['phot_g_mean_mag','phot_bp_mean_mag','phot_rp_mean_mag']].plot.box()